In [1]:
# 采用全量数据训练embedding 只训练用户属性去表达广告属性，且广告属性以'N'补全所有句子（没有点击行为）
# 构建句子中加入slotid_nettypeid
import pandas as pd
import numpy as np
import gc
from base import Cache
from tqdm import tqdm
from gensim.models import Word2Vec
import sys
from multiprocessing import Pool
from sklearn.preprocessing import LabelEncoder

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('precision', 5)
pd.set_option('display.float_format', lambda x: '%.5f' % x)
pd.set_option('max_colwidth', 200)
pd.set_option('display.width', 5000)

# # log
# class Logger(object):
#     def __init__(self, fileN="Default.log"):
#         self.terminal = sys.stdout
#         self.log = open(fileN, "a", encoding='utf-8')

#     def write(self, message):
#         self.terminal.write(message)
#         self.log.write(message)

#     def flush(self):
#         pass
# sys.stdout = Logger("zlh0918log.txt")

def reduce_mem(df, use_float16=False):
    start_mem = df.memory_usage().sum() / 1024**2
    tm_cols = df.select_dtypes('datetime').columns
    for col in df.columns:
        if col in tm_cols:
            continue
        col_type = df[col].dtypes
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(
                        np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(
                        np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(
                        np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(
                        np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if use_float16 and c_min > np.finfo(
                        np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(
                        np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
    end_mem = df.memory_usage().sum() / 1024**2
    print('{:.2f} Mb, {:.2f} Mb ({:.2f} %)'.format(
        start_mem, end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df
    
def gen_list_df(data_):
    data = data_.copy()
#     onehotlist = ['age','city_rank','career','gender']
    feature1,feature2 = list(data.columns)[-2],list(data.columns)[-1]# 主键,emb字段
    print(f'{feature1},{feature2} start!')
    set1 = set(data.query('label<2')[feature1]).intersection(set(data.query('label==2')[feature1]))
    set2 = set(data.query('label<2')[feature2]).intersection(set(data.query('label==2')[feature2]))
    data.loc[~data[feature1].isin(set1),feature1]=-1
    data.loc[~data[feature2].isin(set2),feature2]=-1# unk 词input不在字典里的词都是 -1 每点击过1的词 和没见过的词都用-1替代生成embedding
    setfeature1 = pd.DataFrame(list(set(data[feature1])),columns=[feature1])# 所有feature1
    setkey = set(data[feature2].astype(str))
    data[feature2] = data[feature2].astype(str)+'!'+data['slot_id_net_type'].astype(str)+'Side'# 拼接side information
    list_df=data.query('label==1').groupby(feature1)[feature2].apply(lambda x: ('!'.join(list(x))).split('!')).\
            rename(f'list_{feature2}').reset_index()
    list_df = setfeature1.merge(list_df,on=feature1,how='left')
    list_df[f'list_{feature2}']=list_df[f'list_{feature2}'].map(lambda x:x if isinstance(x,list) else ['N'])
    list_df_list = list_df[f'list_{feature2}'].values.tolist()
    print('w2v start!')
    model = Word2Vec(
    list_df_list,
    size=64,
    window=10000,# 增加了10倍
    workers=5,
    min_count=1,
    sg=0,
    hs=0,
    negative=5,
    iter=5,
    seed=0)
    
    emb_matrix = []
    for seq in list_df_list:
        vec = []
        for w in seq:
            # 只看主键
            if w in setkey or w=='N':
                if w in model.wv:
                    vec.append(model.wv[w])
        if len(vec) > 0:
            emb_matrix.append(np.mean(vec, axis=0))
        else:
            emb_matrix.append([0] * 64)
            
    emb_matrix = np.array(emb_matrix)
    
    dict_feature1 = {}
    for index,key in enumerate(list(list_df[feature1])):
        dict_feature1[key]=emb_matrix[index,:]# feature1的字典 key是int
    dict_feature2 = {}
    for index,key in enumerate(list(setkey)):
        if key!='N':
            dict_feature2[int(key)]=model.wv[key]# feature2的字典 key是str 还是转int吧
    dict_key_value = {}
    dict_key_value['key']=dict_feature1
    dict_key_value['value']=dict_feature2
    
    Cache.cache_data(dict_key_value, nm_marker=f'EMB_TARGET_DICT_{feature1}__{feature2}_w2v__add_slotidnettype_All')
    print(emb_matrix.shape,list_df.shape)
    print(f'{feature1},{feature2}done!')

from multiprocessing import Pool
if __name__ == '__main__':
    print('start!')
    data = Cache.reload_cache('CACHE_dataall0831_fillna.pkl')
    print(data.shape)
    data['label']=data['label'].fillna(2).astype(int)# mask
    gc.collect()
    data['slot_id_net_type'] = data['slot_id'].values*10+data['net_type'].values# 作为场景交叉项 直接onehot
#     # 生成一个emb matrix
#     le = LabelEncoder()
#     data['slot_id_net_type'] = le.fit_transform(data['slot_id_net_type']).astype(int)
#     n_data_matrix= np.eye(data['slot_id_net_type'].nunique())[data['slot_id_net_type'].values]
#     n_data_matrix = n_data_matrix.astype(np.int32)
#     np.save('EMB_slot_id_net_type_embedding_matrix_All.npy',n_data_matrix)
    print('w2v start!')
    # 生成一个emb matrix
    user_fe_list = ['age','career','gender','city','device_name','residence','emui_dev','city_rank']# 'city_rank'
    item_fe_list = ['task_id','adv_id','adv_prim_id','tags','spread_app_id']
    poc_feature_list = []
    for i,vari in enumerate(user_fe_list):
        for j,varj in enumerate(item_fe_list):
#             poc_feature_list.append([vari,varj])# key只用广告id，比较多
            poc_feature_list.append([varj,vari])
    print(len(poc_feature_list))
    datalist = []
    for item in poc_feature_list:
        datalist.append(data[['uid','index','pt_d','label','slot_id_net_type',item[0],item[1]]])# 拆分数据集
    with Pool(5) as p:
        result = p.map(gen_list_df, datalist)


start!


[2020-09-22 01:32:10] - __init__.py[line:126] - INFO: Successfully Reload: /home/zhangqibot/proj/digix/zlh/cached_data/CACHE_dataall0831_fillna.pkl


(42907133, 38)
w2v start!
40
task_id,age start!
adv_prim_id,age start!
spread_app_id,age start!
adv_id,career start!
tags,career start!
w2v start!


[2020-09-22 01:54:58] - word2vec.py[line:1399] - INFO: collecting all words and their counts
[2020-09-22 01:54:58] - word2vec.py[line:1384] - INFO: PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
[2020-09-22 01:54:58] - word2vec.py[line:1407] - INFO: collected 55 word types from a corpus of 2890976 raw words and 50 sentences
[2020-09-22 01:54:58] - word2vec.py[line:1458] - INFO: Loading a fresh vocabulary
[2020-09-22 01:54:58] - word2vec.py[line:1482] - INFO: effective_min_count=1 retains 55 unique words (100% of original 55, drops 0)
[2020-09-22 01:54:58] - word2vec.py[line:1488] - INFO: effective_min_count=1 leaves 2890976 word corpus (100% of original 2890976, drops 0)
[2020-09-22 01:54:58] - word2vec.py[line:1547] - INFO: deleting the raw counts dictionary of 55 items
[2020-09-22 01:54:58] - word2vec.py[line:1550] - INFO: sample=0.001 downsamples 24 most-common words
[2020-09-22 01:54:58] - word2vec.py[line:1553] - INFO: downsampling leaves estimated 478622 word c

w2v start!


...

[2020-09-22 05:26:24] - base_any2vec.py[line:348] - INFO: worker thread finished; awaiting finish of 3 more threads
[2020-09-22 05:26:25] - base_any2vec.py[line:1291] - INFO: EPOCH 5 - PROGRESS: at 97.50% examples, 3038 words/s, in_qsize 2, out_qsize 1
[2020-09-22 05:26:25] - base_any2vec.py[line:348] - INFO: worker thread finished; awaiting finish of 2 more threads
[2020-09-22 05:26:25] - base_any2vec.py[line:348] - INFO: worker thread finished; awaiting finish of 1 more threads
[2020-09-22 05:26:25] - base_any2vec.py[line:348] - INFO: worker thread finished; awaiting finish of 0 more threads
[2020-09-22 05:26:25] - base_any2vec.py[line:1332] - INFO: EPOCH - 5 : training on 2890976 raw words (273708 effective words) took 83.7s, 3270 effective words/s
[2020-09-22 05:26:25] - base_any2vec.py[line:1368] - INFO: training on a 14454880 raw words (1368484 effective words) took 425.5s, 3216 effective words/s
[2020-09-22 05:26:28] - __init__.py[line:111] - INFO: Cache Successfully! File name:

(80, 64) (80, 2)
adv_prim_id,city_rankdone!
w2v start!


[2020-09-22 05:29:41] - word2vec.py[line:1399] - INFO: collecting all words and their counts
[2020-09-22 05:29:41] - word2vec.py[line:1384] - INFO: PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
[2020-09-22 05:29:41] - word2vec.py[line:1407] - INFO: collected 52 word types from a corpus of 2890976 raw words and 50 sentences
[2020-09-22 05:29:41] - word2vec.py[line:1458] - INFO: Loading a fresh vocabulary
[2020-09-22 05:29:41] - word2vec.py[line:1482] - INFO: effective_min_count=1 retains 52 unique words (100% of original 52, drops 0)
[2020-09-22 05:29:41] - word2vec.py[line:1488] - INFO: effective_min_count=1 leaves 2890976 word corpus (100% of original 2890976, drops 0)
[2020-09-22 05:29:41] - word2vec.py[line:1547] - INFO: deleting the raw counts dictionary of 52 items
[2020-09-22 05:29:41] - word2vec.py[line:1550] - INFO: sample=0.001 downsamples 21 most-common words
[2020-09-22 05:29:41] - word2vec.py[line:1553] - INFO: downsampling leaves estimated 428286 word c

[2020-09-22 05:31:48] - base_any2vec.py[line:1291] - INFO: EPOCH 3 - PROGRESS: at 68.00% examples, 3966 words/s, in_qsize 9, out_qsize 0
[2020-09-22 05:31:51] - base_any2vec.py[line:1291] - INFO: EPOCH 3 - PROGRESS: at 78.00% examples, 3917 words/s, in_qsize 9, out_qsize 0
[2020-09-22 05:31:53] - base_any2vec.py[line:1291] - INFO: EPOCH 3 - PROGRESS: at 86.00% examples, 4143 words/s, in_qsize 7, out_qsize 0
[2020-09-22 05:31:59] - base_any2vec.py[line:1291] - INFO: EPOCH 3 - PROGRESS: at 90.00% examples, 3796 words/s, in_qsize 5, out_qsize 0
[2020-09-22 05:31:59] - base_any2vec.py[line:348] - INFO: worker thread finished; awaiting finish of 4 more threads
[2020-09-22 05:32:02] - base_any2vec.py[line:1291] - INFO: EPOCH 3 - PROGRESS: at 94.00% examples, 3962 words/s, in_qsize 3, out_qsize 1
[2020-09-22 05:32:02] - base_any2vec.py[line:348] - INFO: worker thread finished; awaiting finish of 3 more threads
[2020-09-22 05:32:03] - base_any2vec.py[line:1291] - INFO: EPOCH 3 - PROGRESS: at 9

(50, 64) (50, 2)
spread_app_id,city_rankdone!


In [2]:
poc_feature_list

[['task_id', 'age'],
 ['adv_id', 'age'],
 ['adv_prim_id', 'age'],
 ['tags', 'age'],
 ['spread_app_id', 'age'],
 ['task_id', 'career'],
 ['adv_id', 'career'],
 ['adv_prim_id', 'career'],
 ['tags', 'career'],
 ['spread_app_id', 'career'],
 ['task_id', 'gender'],
 ['adv_id', 'gender'],
 ['adv_prim_id', 'gender'],
 ['tags', 'gender'],
 ['spread_app_id', 'gender'],
 ['task_id', 'city'],
 ['adv_id', 'city'],
 ['adv_prim_id', 'city'],
 ['tags', 'city'],
 ['spread_app_id', 'city'],
 ['task_id', 'device_name'],
 ['adv_id', 'device_name'],
 ['adv_prim_id', 'device_name'],
 ['tags', 'device_name'],
 ['spread_app_id', 'device_name'],
 ['task_id', 'residence'],
 ['adv_id', 'residence'],
 ['adv_prim_id', 'residence'],
 ['tags', 'residence'],
 ['spread_app_id', 'residence'],
 ['task_id', 'emui_dev'],
 ['adv_id', 'emui_dev'],
 ['adv_prim_id', 'emui_dev'],
 ['tags', 'emui_dev'],
 ['spread_app_id', 'emui_dev'],
 ['task_id', 'city_rank'],
 ['adv_id', 'city_rank'],
 ['adv_prim_id', 'city_rank'],
 ['tags',